In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2, 4"

In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
import gc
xgb.__version__

'1.6.0rc1'

In [4]:
!nvidia-smi

Fri Apr 22 11:09:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    62W / 275W |    902MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM-80GB (UUID: GPU-a19c00c3-2832-fe38-1c43-c18db3e909da)
GPU 1: NVIDIA A100-SXM-80GB (UUID: GPU-58b97c92-e879-49d3-85b5-1d9615f10873)
GPU 2: NVIDIA A100-SXM-80GB (UUID: GPU-d21cfed4-2e1a-f313-839c-ea008aca027a)
GPU 3: NVIDIA DGX Display (UUID: GPU-bda05ff2-94b1-c885-24f7-586c8f86b408)
GPU 4: NVIDIA A100-SXM-80GB (UUID: GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac)


In [6]:
train_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-mar-2021/xgtrain.csv')
test_x = dd.read_csv('../../TPS_2021/input/tabular-playground-series-mar-2021/xgval.csv')

In [7]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [8]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-a19c00c3-2832-fe38-1c43-c18db3e909da",
                                                 "GPU-58b97c92-e879-49d3-85b5-1d9615f10873",
                                                 "GPU-d21cfed4-2e1a-f313-839c-ea008aca027a",
                                                 "GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac"])
client = Client(cluster)

/home/tunguz/.local/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38275 instead
  warnings.warn(
2022-04-22 11:10:43,297 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-22 11:10:43,304 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-22 11:10:43,320 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-04-22 11:10:43,345 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [9]:
!nvidia-smi

Fri Apr 22 11:10:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    61W / 275W |   1313MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [11]:
train_x.head()

,cat0,cat1,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,...,cont1,cont10,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9
0,1,12,285,0,0,0,0,1,3,3,...,0.158880,0.260808,0.241366,0.316379,0.597810,0.226669,0.568740,0.168526,0.263084,0.349155
1,0,6,179,1,0,0,1,3,1,2,...,0.491503,0.472800,0.441523,0.239773,0.255564,0.782164,0.319098,0.376074,0.373659,0.613428
2,0,10,58,0,1,0,1,3,1,3,...,0.432082,0.322575,0.430172,0.272920,0.820504,0.557219,0.457611,0.441664,0.721024,0.271835
3,0,10,174,0,0,0,1,3,3,3,...,0.752207,0.585071,0.761392,0.778366,0.800075,0.772982,0.730863,0.710302,0.745487,0.971073
4,0,8,79,0,0,0,1,1,1,3,...,0.213323,0.394398,0.354421,0.619004,0.794750,0.307437,0.487469,0.361823,0.375954,0.459699


In [12]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [13]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [14]:
num_round = 1000

In [15]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()

    roc = roc_auc_score(test_y, predictions)
    
    return roc

In [16]:
study = optuna.create_study(direction='maximize')

[I 2022-04-22 11:13:42,760] A new study created in memory with name: no-name-c75d49f1-7368-4e77-a341-7ca3644787a7


In [17]:
%%time
study.optimize(objective, n_trials=2)

[11:13:54] task [xgboost.dask]:tcp://127.0.0.1:45461 got new rank 0
[I 2022-04-22 11:14:06,432] Trial 0 finished with value: 0.8932670758767047 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.18840113107273218, 'alpha': 0.11047616109990124, 'colsample_bytree': 0.41350622614038546, 'subsample': 0.8321231620941887, 'learning_rate': 0.014894065320899888, 'max_depth': 17, 'min_child_weight': 242, 'eval_metric': 'logloss'}. Best is trial 0 with value: 0.8932670758767047.
[11:14:06] task [xgboost.dask]:tcp://127.0.0.1:45461 got new rank 0
[I 2022-04-22 11:14:22,452] Trial 1 finished with value: 0.8948835728983169 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.9659618073639074, 'alpha': 6.404986127865654, 'colsample_bytree': 0.47499097357270176, 'subsample': 0.7653040460039003, 'learning_rate': 0.018434759213717084, 'max_depth': 13, 'min_child_weight': 92, 'eval_metric': 'logloss'}. Best is trial 1 with value: 0.

CPU times: user 4.65 s, sys: 1.07 s, total: 5.72 s
Wall time: 28.6 s


In [18]:
%%time
study.optimize(objective, n_trials=50)

[11:15:25] task [xgboost.dask]:tcp://127.0.0.1:45461 got new rank 0
[I 2022-04-22 11:15:27,736] Trial 2 finished with value: 0.8766049943668826 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.09748369438586474, 'alpha': 0.5381672217621859, 'colsample_bytree': 0.9869600880539531, 'subsample': 0.7784857626037505, 'learning_rate': 0.006267492355578035, 'max_depth': 3, 'min_child_weight': 90, 'eval_metric': 'logloss'}. Best is trial 1 with value: 0.8948835728983169.
[11:15:27] task [xgboost.dask]:tcp://127.0.0.1:45461 got new rank 0
[I 2022-04-22 11:15:36,937] Trial 3 finished with value: 0.8953616189747597 and parameters: {'objective': 'binary:logistic', 'tree_method': 'gpu_hist', 'lambda': 0.6986580767014361, 'alpha': 0.003233407970929253, 'colsample_bytree': 0.3906229930991583, 'subsample': 0.9090099454724018, 'learning_rate': 0.05550288380306352, 'max_depth': 17, 'min_child_weight': 253, 'eval_metric': 'logloss'}. Best is trial 3 with value: 0.89

CPU times: user 32 s, sys: 18.4 s, total: 50.4 s
Wall time: 15min 11s


/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 3431089 should have a CUDA context assigned to device GPU-e3b349d7-ac6c-77ab-3564-ed9d05d50bac, but instead the CUDA context is on device 4. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-04-22 15:25:32,495 - distributed.diskutils - ERROR - Failed to remove '/home/tunguz/Programming/Kaggle/TPS_03_2022/scripts/dask-worker-space/worker-e9aky6c6' (failed in <built-in function lstat>): [Errno 2] No such file or directory: '/home/tunguz/Programming/Kaggle/TPS_03_2022/scripts/dask-worker-space/worker-e9aky6c6'
/home/tunguz/.local/lib/python3.8/site-packages/dask_cuda/initialize.py:57: UserWarning: Worker with process ID 3431084 should have a CUDA context assigned to device GPU